In [18]:
import numpy as np

def sigmoid(x):
    """Sigmoid function."""
    return 1 / (1 + np.exp(-x))

def irls(X, y, max_iterations=100, tolerance=1e-6):
    """
    Iteratively Reweighted Least Squares (IRLS) algorithm for logistic regression.
    
    Parameters:
    X (array-like, shape=(n_samples, n_features)): Training data.
    y (array-like, shape=(n_samples,)): Target labels.
    max_iterations (int): Maximum number of iterations. Default is 100.
    tolerance (float): Convergence criterion. Default is 1e-6.
    
    Returns:
    beta (array-like, shape=(n_features,)): Estimated coefficients.
    """
    # Initialize beta with zeros
    beta = np.zeros(X.shape[1])
    
    # Loop until convergence or maximum iterations reached
    for i in range(max_iterations):
        # Compute probabilities using current coefficients
        p = sigmoid(X @ beta)
        
        # Compute diagonal weight matrix
        W = np.diag(p * (1 - p))
        
        # Compute gradient and Hessian
        gradient = X.T @ (y - p)
        hessian = X.T @ W @ X
        
        # Update coefficients
        beta_new = beta + np.linalg.inv(hessian) @ gradient
        
        # Check convergence
        if np.max(np.abs(beta_new - beta)) < tolerance:
            break
        
        beta = beta_new
    
    return beta


In [19]:
X = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([0, 1, 0])

beta = irls(X, y)
print(beta)


[ 0.69314718 -0.69314718]
